In [179]:
import numpy as np
import pandas as pd
import datetime
import re



In [177]:
# create a function to clean the tweets
def cleanTwt(twt):
    twt = re.sub("#aapl", 'aapl', twt) # removes the '#' from apple
    twt = re.sub("#AAPL", 'AAPL', twt)
    twt = re.sub("#amzn", 'amzn', twt) # removes the '#' from amazon
    twt = re.sub("#AMZN", 'AMZN', twt)
    twt = re.sub("#googl", 'googl', twt) # removes the '#' from google
    twt = re.sub("#GOOGL", 'GOOGL', twt)
    twt = re.sub("#msft", 'msft', twt) # removes the '#' from microsoft
    twt = re.sub("#MSFT", 'MSFT', twt)
    twt = re.sub("#tsla", 'tsla', twt) # removes the '#' from tesla
    twt = re.sub("#TSLA", 'TSLA', twt)
    twt = re.sub('#[A-Za-z0-9]+', '', twt) # removes any string with a '#'
    twt = re.sub('\\n', '', twt) # removes the '\n' string
    twt = re.sub('https:\/\/\S+', '', twt) # removes any hyperlinks
    return twt

In [142]:
tweetdf = pd.read_csv('tweet.csv')
valuesdf = pd.read_csv('CompanyValues.csv')
corresponddf = pd.read_csv('Company_tweet.csv')

In [143]:
tweets = pd.merge(tweetdf,corresponddf)

In [144]:
corresponddf.groupby(['tweet_id'],as_index=False).size()  #there are tweets with more than one brand

,tweet_id,size
0,550441509175443456,1
1,550441672312512512,1
2,550441732014223360,1
3,550442977802207232,1
4,550443807834402816,2
...,...,...
3717959,1212159765914079234,1
3717960,1212159838882533376,1
3717961,1212160015332728833,1
3717962,1212160410692046849,2


In [145]:
tweets['ticker_symbol'].isna().sum()  #check how many tweets do not have a company assigned

0

In [146]:
tweets["day_date"] = tweets["post_date"].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d'))


In [147]:
valuesdf.sort_values(by=["ticker_symbol","day_date"], inplace = True) 

In [148]:
valuesdf.reset_index()

,index,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
0,2516,AAPL,2010-06-01,37.2614,218455576,37.0986,37.9914,36.9943
1,2515,AAPL,2010-06-02,37.7071,171594061,37.7914,37.8286,37.1907
2,2514,AAPL,2010-06-03,37.5886,162341809,37.8828,37.9357,37.2014
3,2513,AAPL,2010-06-04,36.5664,189045767,36.8871,37.4143,36.3757
4,2512,AAPL,2010-06-07,35.8486,221253336,36.8986,37.0214,35.7928
...,...,...,...,...,...,...,...,...
17523,14467,TSLA,2020-05-22,816.8800,9987475,822.1740,831.7800,812.0000
17524,14466,TSLA,2020-05-26,818.8700,8089736,834.5000,834.6000,815.7050
17525,14465,TSLA,2020-05-27,820.2300,11549530,820.8600,827.7100,785.0000
17526,14464,TSLA,2020-05-28,805.8100,7275774,813.5100,824.7500,801.6900


In [159]:
companies = list(valuesdf["ticker_symbol"].unique())
valuesdf["diff"] = np.nan
df = pd.DataFrame()
for company in companies:
    a = valuesdf[valuesdf['ticker_symbol']==company]
    a["diff"]= a["close_value"].diff(1)
    df = pd.concat([df,a])

C:\Users\cisar\AppData\Local\Temp\ipykernel_26204\3865908871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["diff"]= a["close_value"].diff(1)
C:\Users\cisar\AppData\Local\Temp\ipykernel_26204\3865908871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["diff"]= a["close_value"].diff(1)
C:\Users\cisar\AppData\Local\Temp\ipykernel_26204\3865908871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [168]:
df["per"] = df["diff"]/df["close_value"]*100
df["impact"] = np.nan
df["impact"]= np.where(df["per"] > 0.2,  'Rise', df["impact"])
df["impact"]= np.where(df["per"] < -0.2,  'Drop', df["impact"])
df["impact"]= np.where((df["per"] <= 0.2) & (df["per"] >= -0.2 ),  'Neutral', df["impact"])
df["impact"]= np.where(df["per"]==0,  'Irrelevant', df["impact"])

In [169]:
df.groupby(['impact'],as_index=False).size()

,impact,size
0,Drop,5730
1,Irrelevant,3448
2,Neutral,1840
3,Rise,6504
4,nan,6


In [172]:
df = df.drop(['diff'],axis=1)

In [180]:
final = tweets.merge(df, on = ["ticker_symbol","day_date"])
final["body"] = final["body"].apply(cleanTwt)

In [181]:
final

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,per,impact
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,0.000000,Irrelevant
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,0.000000,Irrelevant
2,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,0.000000,Irrelevant
3,550443808606126081,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,0.000000,Irrelevant
4,550443809700851716,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01,110.38,41304780,112.82,113.13,110.21,0.000000,Irrelevant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4336440,1212154334600818689,thequ4nt,1577835088,"[Yahoo Finance Video - December 31, 2019] $AMZ...",0,1,1,AMZN,2019-12-31,1847.84,2510380,1842.00,1853.26,1832.23,0.051411,Neutral
4336441,1212155226377117697,DentonBlackwell,1577835301,@Ryzenn $AAPL has to have a lights out ER to k...,0,0,1,AMZN,2019-12-31,1847.84,2510380,1842.00,1853.26,1832.23,0.051411,Neutral
4336442,1212156775178416128,ITRADEVOL,1577835670,Is there a public database or website where I ...,0,0,2,AMZN,2019-12-31,1847.84,2510380,1842.00,1853.26,1832.23,0.051411,Neutral
4336443,1212157343351238656,naashonomics,1577835806,$FB $AMZN $GOOGL $NFLX 2019,0,0,0,AMZN,2019-12-31,1847.84,2510380,1842.00,1853.26,1832.23,0.051411,Neutral


In [175]:
final.groupby(['impact'],as_index=False).size()

,impact,size
0,Drop,1512914
1,Irrelevant,639815
2,Neutral,451381
3,Rise,1732335


In [182]:
final.to_csv('trend_dataset.csv', index=False)